<a href="https://colab.research.google.com/github/Jvnc0503/DL-Lab1-Animal-Sounds/blob/main/Laboratorio_1_Animal_Sounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

drive_path = "/content/drive/MyDrive"
dataset_path = f"{drive_path}/Animal Sounds"
local_path = "/content/data"
audio_path = f"{drive_path}/Sounds"

Mounted at /content/drive


In [ ]:
!mkdir -p {audio_path}
!7z x "{dataset_path}/train.7z" -o"{local_path}"
!7z x "{dataset_path}/test.7z" -o"{local_path}"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Animal Sounds/                                               1 file, 7069498436 bytes (6742 MiB)

Extracting archive: /content/drive/MyDrive/Animal Sounds/train.7z
--
Path = /content/drive/MyDrive/Animal Sounds/train.7z
Type = 7z
Physical Size = 7069498436
Headers Size = 727932
Method = LZMA2:23
Solid = +
Blocks = 1

  0%      0% - train/INCT17_20191113_040000_0_3.wav                                             0% 31 - train/INCT17_20191113_040000_38_41.wav                                       

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from typing import Optional
from tqdm import tqdm

def process_dataset(input_folder: Path, output_folder: Path):
    # Crear carpeta de salida si no existe
    output_folder.mkdir(parents=True, exist_ok=True)

    files = list((input_folder / "train").glob("*.wav")) + list((input_folder / "test").glob("*.wav"))

    # Limitar el número de archivos si es necesario
    #if limit is not None:
    #    files = files[:limit]

    total_files = len(files)

    print(f"--- Iniciando procesamiento de {total_files} audios ---")
    print(f"Config: fmin=250, fmax=10000, 224x224px")

    for audio in tqdm(files):
        try:
            # --- 1. CARGA Y CONFIGURACIÓN ---
            y, sr = librosa.load(audio)
            n_fft = 2048
            hop_length = 295
            n_mels = 224

            bias = 30
            time_constant = 0.1
            #image = output_folder / (audio.stem + f"(bias={bias})" + audio.suffix + ".png")
            #image = output_folder / (audio.stem + f"(time_constant={time_constant})" + audio.suffix + ".png")
            image = output_folder / (audio.stem + ".png")
            #print(image)

            # --- 2. MEL SPECTROGRAM (Tus nuevos límites) ---
            spectrogram = librosa.feature.melspectrogram(
                y=y,
                sr=sr,
                n_fft=n_fft,
                hop_length=hop_length,
                n_mels=n_mels,
                fmin=250,    # Tu ajuste: Elimina rumble grave
                fmax=10000   # Tu ajuste: Elimina hiss agudo
            )

            # --- 3. PCEN (Normalización) ---
            # Mantengo bias=10 (estándar) para no oscurecer demasiado si ya cortamos frecuencias.
            # Si aún ves mucho ruido, puedes subir bias a 20 o 30.
            spectrogram_pcen = librosa.pcen(
                spectrogram * (2**31),
                time_constant=time_constant,
                gain=0.8,
                bias=bias,
                power=0.25,
                eps=1e-6
            )

            # --- 4. PIXEL PERFECT CROP (Ancho) ---
            # Objetivo: 224 columnas exactas
            target_width = 224

            # Si sobra, cortamos
            if spectrogram_pcen.shape[1] > target_width:
                spectrogram_pcen = spectrogram_pcen[:, :target_width]

            # Si falta (audio < 3s), rellenamos con silencio (padding a la derecha)
            elif spectrogram_pcen.shape[1] < target_width:
                pad_width = target_width - spectrogram_pcen.shape[1]
                spectrogram_pcen = np.pad(spectrogram_pcen, ((0,0), (0, pad_width)), mode='constant')

            # --- 5. EXPORTACIÓN ---
            # plt.imsave es la clave para no tener bordes ni ejes
            plt.imsave(
                image,
                spectrogram_pcen,
                cmap='magma',
                origin='lower',
                format='png'
            )

        except Exception as e:
            print(f"Error procesando {audio}: {e}")

    print("--- ¡Procesamiento completado! ---")

drive_path = Path(drive_path)
local_path = Path(local_path)
spectrogram_path = drive_path / "Spectrograms"

process_dataset(local_path, spectrogram_path)

--- Iniciando procesamiento de 93378 audios ---
Config: fmin=250, fmax=10000, 224x224px


100%|██████████| 93378/93378 [2:17:10<00:00, 11.35it/s]


--- ¡Procesamiento completado! ---


In [ ]:
from google.colab import drive
from pathlib import Path
drive.mount("/content/drive")

drive_path = Path("/content/drive/MyDrive")
spectrogram_path = drive_path / "Spectrograms"
targets_path = drive_path / "Animal Sounds" / "train.csv"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

# --- CONFIGURACIÓN ---
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 15 # Multi-label suele requerir un poco más de tiempo
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- DATASET MULTI-LABEL ---
class MultiLabelDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = Path(img_dir)
        self.transform = transform

        # Obtenemos los nombres de las especies (todas las columnas menos 'filename')
        self.species_columns = df.columns[1:]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 1. Nombre del archivo
        row = self.df.iloc[idx]
        fname = row['filename']

        # 2. Vector de Etiquetas (Targets)
        # Tomamos desde la columna 1 hasta el final.
        # Convertimos a float32 porque BCEWithLogitsLoss lo exige.
        labels = row[1:].values.astype('float32')

        # 3. Carga de Imagen
        img_name = Path(fname).stem + ".png" # Asumimos que guardaste como .png
        img_path = self.img_dir / img_name

        try:
            image = Image.open(img_path).convert('L') # Escala de grises
        except FileNotFoundError:
            image = Image.new('L', (224, 224)) # Imagen negra de seguridad

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(labels)

# --- PREPARACIÓN DE DATOS ---
# 1. Cargar CSV
df = pd.read_csv(targets_path)

# Calcular número de clases automáticamente basado en tus columnas
NUM_CLASSES = len(df.columns) - 1
print(f"Detectadas {NUM_CLASSES} especies para clasificación.")

# 2. Split (Estratificar en multi-label es complejo, usaremos random simple por ahora)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# 3. Transformaciones
data_transforms = transforms.Compose([
    transforms.ToTensor(),
])

# 4. DataLoaders
train_dataset = MultiLabelDataset(train_df, spectrogram_path, transform=data_transforms)
val_dataset = MultiLabelDataset(val_df, spectrogram_path, transform=data_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
class AudioNet(nn.Module):
    def __init__(self, num_classes):
        super(AudioNet, self).__init__()

        # Bloques convolucionales (Feature Extraction)
        self.features = nn.Sequential(
            # Bloque 1
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            # Bloque 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            # Bloque 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            # Bloque 4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.flatten = nn.Flatten()

        # Clasificador
        # Entrada: 256 canales * 14 * 14
        self.classifier = nn.Sequential(
            nn.Linear(256 * 14 * 14, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
            # NOTA: No usamos Sigmoid aquí porque BCEWithLogitsLoss lo incluye dentro
            # para mayor estabilidad numérica.
        )

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

model = AudioNet(num_classes=NUM_CLASSES).to(DEVICE)